# Mediator

> Facilitating communication between components

Let's create a chat room. We can think of a chat room as a component (in a system) where people can join in and leave at any moment. Users don't necessarily have to be aware of one another unless they're sending a direct message, but if they're sending messages to the room, then it doesn't really matter how many of them there are.

Let's being by defining the basic components that we will work with:

In [1]:
class Person:
    """User in the chat room. Has a username, a chat log and a room associated to it"""
    def __init__(self, name):
        self.name = name
        self.chat_log = []
        self.room = None

class ChatRoom:
    """Essentially a list of users"""
    def __init__(self):
        self.people = []

We will now expand upon `ChatRoom` by allowing people to join the room. We will also define a `broadcast` method to send messages to the room.

> For simplicity's sake we will not be implementing a `leave` method, but obviously we want users to be able to freely leave the room at any time.

In [2]:
class ChatRoom:
    def __init__(self):
        self.people = []

    def join(self, person):
        """Adds the user to the people list and gives the user a reference to the chat room"""
        join_msg = f'{person.name} joins the chat'
        self.broadcast('room', join_msg)
        person.room = self
        self.people.append(person)
    
    def broadcast(self, source, message):
        """Send a message to everyone in the room"""
        for p in self.people:
            if p.name != source:
                p.receive(source, message)

The `broadcast` method is saying that every `Person` in the room must receive a message, so let's add that capability to `Person`:

In [3]:
class Person:
    def __init__(self, name):
        self.name = name
        self.chat_log = []
        self.room = None

    def receive(self, sender, message):
        s = f'{sender}: {message}'
        print(f'[{self.name}\'s chat session] {s}') # We specify the Person's chat session
        self.chat_log.append(s)

Another common feature of chat rooms is the ability to directly message another user. We want to make sure that if user A sends a message to user B who just left the room, our code will be able to handle it and not crash.

In [4]:
class ChatRoom:
    def __init__(self):
        self.people = []

    def join(self, person):
        """Adds the user to the people list and gives the user a reference to the chat room"""
        join_msg = f'{person.name} joins the chat'
        self.broadcast('room', join_msg)
        person.room = self
        self.people.append(person)
    
    def broadcast(self, source, message):
        """Send a message to everyone in the room"""
        for p in self.people:
            if p.name != source:
                p.receive(source, message)

    def message(self, source, destination, message):
        """Direct messaging between users"""
        for p in self.people:
            if p.name == destination:
                p.receive(source, message) # we only deliver the message if the user is present in the chat room

The chat room can now handle direct messages but `Person` still does not know how to send one, so let's enable it. Since we're at it, we'll also enable sending messages to the chat room.

In [5]:
class Person:
    def __init__(self, name):
        self.name = name
        self.chat_log = []
        self.room = None

    def receive(self, sender, message):
        s = f'{sender}: {message}'
        print(f'[{self.name}\'s chat session] {s}')
        self.chat_log.append(s)

    def private_message(self, who, message):
        self.room.message(self.name, who, message) # Note that we're sending the message to the room rather than to another user directly
    
    def say(self, message):
        self.room.broadcast(self.name, message)

In this scenario, `ChatRoom` behaves as the **mediator** between users. Users don't send direct messages directly between them, but rather, they send them to `ChatRoom` and it takes care of delivering the private messages.

We now have everything in place. Let's simulate a chat room:

In [6]:
room = ChatRoom()

john = Person('John')
jane = Person('Jane')

room.join(john)
room.join(jane)

john.say('hi room')
jane.say('oh, hey john')

simon = Person('Simon')
room.join(simon)
simon.say('hi everyone!')

jane.private_message('Simon', 'glad you could join us!')

[John's chat session] room: Jane joins the chat
[Jane's chat session] John: hi room
[John's chat session] Jane: oh, hey john
[John's chat session] room: Simon joins the chat
[Jane's chat session] room: Simon joins the chat
[John's chat session] Simon: hi everyone!
[Jane's chat session] Simon: hi everyone!
[Simon's chat session] Jane: glad you could join us!


Note that there is no *John joins the chat* message because John is the first user to join the chat, thus there's nobody else to broadcast the join message to. When Simon joins, both John and Jane receive the broadcast message.